In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


names = []
for x in range(69):
  names.append(x)

def load_housing_data():
  return pd.read_csv("/content/Citizen_Satisfaction_Survey.csv", names = names, header = 0)


In [17]:
survey = load_housing_data()
survey.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68
0,1,2007,23456,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,NaN,NaN,NaN,Satisfied,Very Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Dissatisfied,Dissatisfied,Very Satisfied,Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,They should reduce the traffic flow problems,No,No,No,Yes,No,No,Yes,NaN,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,Strongly Agree,Agree,NaN,NaN,NaN,NaN,Owns/Has mortgage,12,No,Suffolk,45 to 54,NaN,White (Caucasian),Beyond high school,Yes,"$100,000 to $124,999",Male
1,2,2007,23455,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,NaN,NaN,NaN,Satisfied,Satisfied,Satisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,I think everything is fine,No,No,No,Yes,No,No,Yes,NaN,Agree,Agree,Agree,Strongly Agree,NaN,Strongly Agree,Agree,NaN,NaN,NaN,NaN,Rents,15,No,Chesapeake,45 to 54,NaN,African American,High school or less,No,"$20,000 to $39,999",Male
2,3,2007,23455,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,Very Satisfied,Satisfied,Very Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,Very Satisfied,NaN,NaN,NaN,Satisfied,Dissatisfied,Very Satisfied,Very Satisfied,Dissatisfied,Very Dissatisfied,Dissatisfied,Very Dissatisfied,Dissatisfied,Very Satisfied,Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,The City should create/preserve more green spaces,No,Yes,Yes,Yes,Yes,No,Yes,NaN,Strongly Agree,Agree,Disagree,Strongly Agree,NaN,Strongly Agree,Strongly Agree,NaN,NaN,NaN,NaN,Rents,35,No,Chesapeake,45 to 54,NaN,White (Caucasian),High school or less,No,"Under $20,000",Female
3,4,2007,23452,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,NaN,NaN,NaN,Dissatisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,They should focus more on regionalism and regi...,No,No,Yes,No,No,Yes,No,NaN,Strongly Agree,Agree,Agree,Strongly Agree,NaN,Strongly Agree,Agree,NaN,NaN,NaN,NaN,Owns/Has mortgage,19,No,Norfolk,45 to 54,NaN,African American,Beyond high school,Yes,"$125,000 or more",Male
4,5,2007,23462,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,NaN,NaN,NaN,Satisfied,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,Satisfied,Very Satisfied,Satisfied,Very Satisfied,I think everything is fine,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Strongly Agree,Agree,Agree,Strongly Agree,NaN,Agree,Agree,NaN,NaN,NaN,NaN,Rents,20,Yes,Virginia Beach (incl. works from home),26 to 34,NaN,African American,Beyond high school,Yes,"$40,000 to $59,999",Female


In [18]:
#the encoders weren't working so i just decided to make my own
def encoder(x, dataset):

  for index in range(len(dataset[x])):
    if dataset[x][index] == "Very Dissatisfied":
      dataset[x][index] = 2
    elif dataset[x][index] == "Dissatisfied":
      dataset[x][index] = 5
    elif dataset[x][index] == "Satisfied":
      dataset[x][index] = 9
    elif dataset[x][index] == "Very Satisfied":
      dataset[x][index] = 14
    else:
      dataset[x][index] = 0
      continue
      
# encoding columns 3-19 
# encoding columns 23-37
def data_clean_up(dataset, columns):
  for item in columns:
    for x in range(item[0],item[1]+1):
      encoder(x, dataset)
      survey[x] = pd.to_numeric(survey[x])



columns = [[3,19],[23,37]]
#decided to convert categorical values before I split the set since we need labels and stuff
data_clean_up(survey, columns)

#copying survey since we're going to need demographic data to do this
survey_demographic = survey
#dropping columns that aren't going to be used
survey = survey.drop([0,1,2,20,21,22,38,46,51,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68], axis=1)
survey["labels"] = survey.sum(axis=1, skipna = True, numeric_only = True)
print(survey["labels"])
#since we've created the labels, I'm going to drop those rows
survey = survey.drop([3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37], axis=1)
labels = pd.DataFrame(data=survey["labels"])
labels = np.log(labels)
survey = survey.drop(["labels"], axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/py

0       360
1       350
2       333
3       314
4       383
       ... 
3023    282
3024    236
3025    268
3026    267
3027    292
Name: labels, Length: 3028, dtype: int64


In [19]:
survey.head()

,39,40,41,42,43,44,45,47,48,49,50,52,53
0,No,No,No,Yes,No,No,Yes,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree
1,No,No,No,Yes,No,No,Yes,Agree,Agree,Agree,Strongly Agree,Strongly Agree,Agree
2,No,Yes,Yes,Yes,Yes,No,Yes,Strongly Agree,Agree,Disagree,Strongly Agree,Strongly Agree,Strongly Agree
3,No,No,Yes,No,No,Yes,No,Strongly Agree,Agree,Agree,Strongly Agree,Strongly Agree,Agree
4,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Strongly Agree,Agree,Agree,Strongly Agree,Agree,Agree


In [20]:
sample_incomplete_rows = survey[survey.isnull().any(axis=1)].head()
sample_incomplete_rows


,39,40,41,42,43,44,45,47,48,49,50,52,53


In [0]:
x_train, x_test, y_train, y_test = train_test_split(survey, labels, test_size=0.20, random_state=42)

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse = False)
x_train_processed = cat_encoder.fit_transform(x_train)


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(x_train_processed, y_train)
lin_pred = lin_reg.predict(x_train_processed)
lin_mse = mean_squared_error(y_train, lin_pred)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.13052557346601282

In [25]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(lin_reg, x_train_processed, y_train,
                                scoring="neg_mean_squared_error", cv=10)
lin_rmse = np.sqrt(-lin_scores)
print(lin_rmse)

[1.27773863e-01 1.44213203e-01 1.28026078e-01 1.28563663e-01
 1.23583160e-01 1.34933864e-01 1.37461367e-01 1.38774122e-01
 1.35244389e-01 2.78175452e+09]


In [26]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(x_train_processed, y_train)


forest_pred = forest_reg.predict(x_train_processed)
forest_mse = mean_squared_error(y_train, forest_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.085646723812786

In [27]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, x_train_processed, y_train,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
print(forest_rmse_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 

[0.14445811 0.15865915 0.13842475 0.13344537 0.1304748  0.1429898
 0.14184544 0.14790421 0.14622145 0.1431286 ]


In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [70,100,150], 'max_features': [3,5,7], "bootstrap":[True, False]}
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(x_train_processed, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'bootstrap': [True, False], 'max

In [29]:
grid_search.best_params_

{'bootstrap': True, 'max_features': 5, 'n_estimators': 150}

In [30]:
cvres = grid_search.cv_results_
# zip() is to combine the column "mean_test_score" with struct "params"
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.14072538119015024 {'bootstrap': True, 'max_features': 3, 'n_estimators': 70}
0.14034049663508308 {'bootstrap': True, 'max_features': 3, 'n_estimators': 100}
0.14022726445463737 {'bootstrap': True, 'max_features': 3, 'n_estimators': 150}
0.14017963241360099 {'bootstrap': True, 'max_features': 5, 'n_estimators': 70}
0.14021775643707887 {'bootstrap': True, 'max_features': 5, 'n_estimators': 100}
0.13999404761541479 {'bootstrap': True, 'max_features': 5, 'n_estimators': 150}
0.14039839338894428 {'bootstrap': True, 'max_features': 7, 'n_estimators': 70}
0.14020412793221734 {'bootstrap': True, 'max_features': 7, 'n_estimators': 100}
0.1401582295952655 {'bootstrap': True, 'max_features': 7, 'n_estimators': 150}
0.14478356266836234 {'bootstrap': False, 'max_features': 3, 'n_estimators': 70}
0.14484922406787729 {'bootstrap': False, 'max_features': 3, 'n_estimators': 100}
0.14469745202805082 {'bootstrap': False, 'max_features': 3, 'n_estimators': 150}
0.14550518698706189 {'bootstrap': False, '

In [31]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances


array([2.47024044e-02, 2.38269681e-02, 1.88370917e-02, 1.86152802e-02,
       2.00522326e-02, 2.02197433e-02, 2.24622085e-02, 1.13731208e-02,
       2.34918840e-02, 2.39081766e-02, 2.36565696e-02, 2.56190910e-02,
       2.56926512e-02, 2.51472287e-02, 2.44161011e-02, 3.25817877e-02,
       3.05085102e-02, 6.17997952e-02, 3.37983438e-03, 2.78083904e-02,
       2.73903151e-02, 5.08447268e-02, 3.64785015e-03, 2.70920438e-02,
       2.59737173e-02, 8.65488833e-05, 1.89114473e-02, 1.29247460e-02,
       1.41501371e-03, 5.91757581e-03, 1.33828863e-02, 7.02781860e-03,
       2.52788988e-02, 1.28131976e-02, 4.26417891e-02, 6.03325661e-03,
       2.26203039e-02, 2.12517602e-02, 4.99338075e-02, 2.83796004e-03,
       3.50583772e-02, 3.36263999e-02, 1.37101256e-03, 4.63050351e-02,
       1.75144423e-02])

In [0]:
labels.describe()